###Extract data from a single file and from a directory of files

In [0]:
df_single = spark.read.parquet("/path/to/single/file.parquet")
display(df_single)

df_dir = spark.read.parquet("/path/to/directory/")
display(df_dir)

###Identify the prefix included after the FROM keyword as the data type

In [0]:
%sql
-- Common prefixes/ways to specify the data source after FROM in Databricks SQL

-- 1. Table Names (qualified or unqualified)
SELECT * FROM my_table;
SELECT * FROM my_schema.my_table;
SELECT * FROM my_catalog.my_schema.my_table;

-- 2. File Paths (external files)
SELECT * FROM 'dbfs:/path/to/my/file.csv';
SELECT * FROM 's3://bucket/path/to/data/';
SELECT * FROM 'abfss://container@storageaccount.dfs.core.windows.net/path/to/data/';

-- 3. Views (temporary or permanent)
SELECT * FROM my_view;
SELECT * FROM my_temp_view;

-- 4. Subqueries
SELECT * FROM (SELECT col1, col2 FROM another_table) AS subquery_alias;

-- 5. Table-valued functions (TVFs)
SELECT * FROM read_files('path/to/files/');

-- 6. VALUES clause (in-memory table)
SELECT * FROM VALUES (1, 'A'), (2, 'B') AS my_data(id, value);

-- 7. AS OF syntax (Delta Lake Time Travel)
SELECT * FROM my_delta_table TIMESTAMP AS OF '2023-01-01';
SELECT * FROM my_delta_table VERSION AS OF 123;
SELECT * FROM my_delta_table@20230101000000000;
SELECT * FROM my_delta_table@v123;

###Create a view, a temporary view, and a CTE as a reference to a file

In [0]:
# Create a permanent view from the parquet file
df_single.write.mode("overwrite").saveAsTable("permanent_view_single")

# Create a temporary view from the parquet file
df_single.createOrReplaceTempView("temp_view_single")

# Use a CTE to reference the file in a SQL query
query = """
WITH cte_single AS (
  SELECT * FROM parquet.`/path/to/single/file.parquet`
)
SELECT * FROM cte_single
"""
df_cte = spark.sql(query)
display(df_cte)

In [0]:
%sql
-- Create a permanent view from the parquet file
CREATE OR REPLACE TABLE permanent_view_single
USING PARQUET
OPTIONS (path "/path/to/single/file.parquet")
AS SELECT * FROM parquet.`/path/to/single/file.parquet`;

-- Create a temporary view from the parquet file
CREATE OR REPLACE TEMP VIEW temp_view_single AS
SELECT * FROM parquet.`/path/to/single/file.parquet`;
    
-- Use a CTE to reference the file in a SQL query
WITH cte_single AS (
  SELECT * FROM parquet.`/path/to/single/file.parquet`
)
SELECT * FROM cte_single;
 

### Identify that tables from external sources are not Delta Lake tables.

In [0]:
# Check if the tables are Delta Lake tables by inspecting their table properties

# Permanent table
permanent_table = "permanent_view_single"
permanent_table_details = spark.sql(f"DESCRIBE DETAIL {permanent_table}")
display(permanent_table_details)

# Temporary view (not a table, so not Delta)
# CTE result is not a table, so not Delta

# For the parquet file directly
from pyspark.sql.utils import AnalysisException

def is_delta_table(path):
    try:
        return spark.read.format("delta").load(path)._jdf.isDelta()
    except AnalysisException:
        return False

is_delta_single = is_delta_table("/path/to/single/file.parquet")
is_delta_dir = is_delta_table("/path/to/directory/")

print(f"Is '/path/to/single/file.parquet' a Delta table? {is_delta_single}")
print(f"Is '/path/to/directory/' a Delta table? {is_delta_dir}")

# Explanation:
# - Tables created from external sources like Parquet files are not Delta Lake tables unless explicitly written in Delta format.
# - Delta tables have transaction logs (_delta_log) and support ACID transactions, schema enforcement, and time travel.
# - Parquet tables lack these features and are considered external, non-Delta tables.
# - DESCRIBE DETAIL on a Delta table shows 'Provider' as 'delta'; for non-Delta, it is 'parquet' or empty.

In [0]:
-- Check if the tables are Delta Lake tables by inspecting their table properties

-- Permanent table
DESCRIBE DETAIL permanent_view_single;

-- Temporary view (not a table, so not Delta)
-- CTE result is not a table, so not Delta

-- For the parquet file directly
DESCRIBE DETAIL parquet.`/path/to/single/file.parquet`;
DESCRIBE DETAIL parquet.`/path/to/directory/`;

###Create a table from a JDBC connection and from an external CSV file

In [0]:
%sql
-- Create a table from a JDBC connection
CREATE OR REPLACE TABLE jdbc_table
USING JDBC
OPTIONS (
  url 'jdbc:postgresql://hostname:5432/database',
  dbtable 'schema.source_table',
  user 'your_username',
  password 'your_password'
);

-- Create a table from an external CSV file
CREATE OR REPLACE TABLE csv_table
USING CSV
OPTIONS (
  path 'dbfs:/path/to/external/file.csv',
  header 'true',
  inferSchema 'true'
)
AS SELECT * FROM csv.`dbfs:/path/to/external/file.csv`;

###Identify how the count_if function and the count where x is null can be used.

In [0]:
%sql
-- Using count_if to count rows where column x is null
SELECT count_if(x IS NULL) AS null_count
FROM permanent_view_single;

-- Using count to count rows where column x is null
SELECT count(*) AS null_count
FROM permanent_view_single
WHERE x IS NULL;

###Identify how the count(row) skips NULL values.

In [0]:
%sql
-- Demonstrate how COUNT(column) skips NULL values

SELECT
  COUNT(x) AS count_non_null_x,      -- counts only non-NULL values in column x
  COUNT(*) AS count_all_rows,        -- counts all rows, including those where x is NULL
  COUNT(CASE WHEN x IS NULL THEN 1 END) AS count_null_x  -- counts only rows where x is NULL
FROM permanent_view_single;

###Deduplicate rows from an existing Delta Lake table.

In [0]:
%sql
-- Deduplicate rows in a Delta Lake table based on all columns

CREATE OR REPLACE TABLE deduplicated_table AS
SELECT DISTINCT *
FROM existing_delta_table;